In [ ]:
# qb model and projections

In [1]:
import pandas as pd

In [2]:
stats = pd.read_csv("ffb_stats.csv")

In [3]:
stats.columns

Index(['Unnamed: 0', 'Player', 'Tm', 'Pos', 'Passing Cmp', 'Passing Att',
       'Passing Yds', 'Passing TD', 'Int', 'Year', 'Rushing Att',
       'Rushing Yds', 'Rushing TD', 'Rushing Fmb', 'Tgt', 'Rec',
       'Receiving Yds', 'Receiving TD', 'Receiving Fmb', 'Touch',
       'Scrimmage Yards', 'PA', 'Yds Allow', 'TO', 'FL', 'Def Int', 'Pen',
       'EXP', 'PR TD', 'KR TD', 'FblTD', 'IntTD', 'Sfty', 'Short FGA',
       'Short FGM', 'Reg FGA', 'Reg FGM', 'Med FGA', 'Med FGM', 'Long FGA',
       'Long FGM', 'Deep FGA', 'Deep FGM', 'XPA', 'XPM', 'Pts', 'Rk'],
      dtype='object')

In [5]:
feats = ['Player','Tm','Pos','Passing Cmp','Passing Att','Passing Yds', 'Passing TD', 'Int','Rushing Att','Rushing Yds', 'Rushing TD','Year','Pts']

In [6]:
ffb_qb = stats[stats['Pos'] == 'QB']

In [7]:
qb = ffb_qb[feats]

In [8]:
qb

,Player,Tm,Pos,Passing Cmp,Passing Att,Passing Yds,Passing TD,Int,Rushing Att,Rushing Yds,Rushing TD,Year,Pts
0,Patrick Mahomes,KAN,QB,383.0,580.0,5097.0,50.0,12.0,60.0,272.0,2.0,2018,424.0
1,Peyton Manning,DEN,QB,450.0,659.0,5477.0,55.0,10.0,32.0,-31.0,1.0,2013,421.0
2,Josh Allen,BUF,QB,409.0,646.0,4407.0,36.0,15.0,122.0,763.0,6.0,2021,416.0
3,Patrick Mahomes,KAN,QB,435.0,648.0,5250.0,41.0,12.0,61.0,358.0,4.0,2022,413.0
4,Lamar Jackson,BAL,QB,265.0,401.0,3127.0,36.0,6.0,176.0,1206.0,7.0,2019,413.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7491,Scott Tolzien,GNB,QB,1.0,1.0,4.0,0.0,0.0,3.0,-3.0,0.0,2015,-3.0
7493,Chad Henne,KAN,QB,0.0,2.0,0.0,0.0,0.0,5.0,-5.0,0.0,2022,-3.0
7498,Curtis Painter,NYG,QB,8.0,16.0,57.0,0.0,2.0,3.0,-2.0,0.0,2013,-3.0
7503,A.J. McCarron,LVR,QB,1.0,3.0,8.0,0.0,0.0,3.0,-2.0,0.0,2018,-3.0


In [9]:
predictors = ['Passing Cmp','Passing Att','Passing Yds', 'Passing TD', 'Int','Rushing Att','Rushing Yds', 'Rushing TD']

In [10]:
train = qb[qb['Year'] < 2022]
test = qb[qb['Year'] == 2022]

In [11]:
from sklearn.linear_model import Ridge

qbm = Ridge(alpha=.1)

In [13]:
qbm.fit(train[predictors], train["Pts"])

Ridge(alpha=0.1)

In [14]:
predictions = qbm.predict(test[predictors])

In [15]:
predictions = pd.DataFrame(predictions, columns=["Pred Pts"], index=test.index)

In [16]:
comb = pd.concat([test[["Player", "Pts"]], predictions], axis=1)

In [17]:
comb

,Player,Pts,Pred Pts
3,Patrick Mahomes,413.0,412.449843
6,Josh Allen,403.0,402.385363
17,Jalen Hurts,372.0,359.903340
21,Joe Burrow,362.0,351.898611
56,Geno Smith,317.0,311.132182
...,...,...,...
6576,Tim Boyle,0.0,-0.267766
6577,C.J. Beathard,0.0,-0.779927
7378,Brandon Allen,-1.0,-0.614424
7387,Nate Sudfeld,-1.0,-1.870123


In [18]:
from sklearn.metrics import mean_squared_error

In [19]:
mean_squared_error(comb["Pts"], comb["Pred Pts"])

26.10118165952004

In [20]:
import pickle

In [22]:
with open('qbm.pkl', 'wb') as model_file:
    pickle.dump(qbm, model_file)

In [38]:
qb_proj = pd.read_csv('qb_projections.csv')

In [40]:
del qb_proj ['Proj Pts']

In [43]:
del qb_proj ['Comp. %']

In [44]:
qb_proj.columns

Index(['Player', 'Team', 'P Att', 'P Comp', 'P Yds', 'P TDs', 'INT', 'Ru Att',
       'Ru Yds', 'Ru TDs'],
      dtype='object')

In [45]:
qb_proj.rename(columns={'P Att': 'Passing Att'}, inplace=True)

In [46]:
qb_proj.rename(columns={'P Comp': 'Passing Cmp'}, inplace=True)

In [47]:
qb_proj.rename(columns={'P Yds': 'Passing Yds'}, inplace=True)

In [48]:
qb_proj.rename(columns={'P TDs': 'Passing TD'}, inplace=True)

In [49]:
qb_proj.rename(columns={'INT': 'Int'}, inplace=True)

In [50]:
qb_proj.rename(columns={'Ru Att': 'Rushing Att'}, inplace=True)

In [51]:
qb_proj.rename(columns={'Ru Yds': 'Rushing Yds'}, inplace=True)

In [52]:
qb_proj.rename(columns={'Ru TDs': 'Rushing TD'}, inplace=True)

In [54]:
qb_proj.rename(columns={'Team': 'Tm'}, inplace=True)

In [55]:
qb_proj

,Player,Tm,Passing Att,Passing Cmp,Passing Yds,Passing TD,Int,Rushing Att,Rushing Yds,Rushing TD
0,Josh Allen,BUF,562,355,4198.7,33.1,13.0,109.7,607.2,6.4
1,Jalen Hurts,PHI,482,312,3706.3,24.6,8.4,158.2,789.8,9.7
2,Patrick Mahomes,KC,601,397,4768.9,36.7,11.1,60.4,325.4,2.3
3,Lamar Jackson,BAL,470,298,3418.3,25.6,10.8,140.4,868.1,5.0
4,Joe Burrow,CIN,573,393,4376.2,33.0,12.6,61.3,197.7,3.3
5,Justin Herbert,LAC,610,408,4439.0,30.0,10.6,57.4,222.3,1.8
6,Justin Fields,CHI,395,245,2803.5,19.6,12.7,162.3,1025.0,6.5
7,Deshaun Watson,CLE,493,320,3681.3,24.9,12.0,93.0,482.6,4.2
8,Trevor Lawrence,JAC,557,360,4008.6,25.9,10.9,63.2,292.2,3.2
9,Kirk Cousins,MIN,591,391,4350.6,29.4,11.8,28.3,102.2,1.2


In [57]:
qb_proj = qb_proj.fillna(0)

In [58]:
qb_pred = qbm.predict(qb_proj[predictors])

In [60]:
qb_pred = pd.DataFrame(qb_pred, columns=["Pred Pts"], index=qb_proj.index)

In [62]:
qb_comb = pd.concat([qb_proj[["Player", "Tm"]], qb_pred], axis=1)

In [63]:
qb_comb

,Player,Tm,Pred Pts
0,Josh Allen,BUF,373.901629
1,Jalen Hurts,PHI,353.837007
2,Patrick Mahomes,KC,363.229510
3,Lamar Jackson,BAL,327.738335
4,Joe Burrow,CIN,326.102301
5,Justin Herbert,LAC,308.209349
6,Justin Fields,CHI,302.626930
7,Deshaun Watson,CLE,297.053527
8,Trevor Lawrence,JAC,290.469380
9,Kirk Cousins,MIN,290.646429


In [64]:
qb_comb = qb_comb.sort_values("Pred Pts", ascending=False)
qb_comb["Pred Rk"] = list(range(1,qb_comb.shape[0]+1))

In [66]:
qb_comb.head(10)

,Player,Tm,Pred Pts,Pred Rk
0,Josh Allen,BUF,373.901629,1
2,Patrick Mahomes,KC,363.229510,2
1,Jalen Hurts,PHI,353.837007,3
3,Lamar Jackson,BAL,327.738335,4
4,Joe Burrow,CIN,326.102301,5
5,Justin Herbert,LAC,308.209349,6
6,Justin Fields,CHI,302.626930,7
7,Deshaun Watson,CLE,297.053527,8
9,Kirk Cousins,MIN,290.646429,9
8,Trevor Lawrence,JAC,290.469380,10


In [67]:
qb_comb.to_csv('proj_qb.csv')